In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.datasport.it/calcio/serie-a2021-2022/marcatori/38.html'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Trova la tabella giusta
table = soup.find('table', class_='marcatori_campionati')

# Prendi tutte le righe dei dati
rows = table.find('tbody').find_all('tr')

data = []
for row in rows:
    cols = row.find_all('td')
    if len(cols) == 3:
        gol = cols[0].text.strip()
        giocatore = cols[1].text.strip()
        squadra = cols[2].text.strip()
        data.append([giocatore, squadra, gol])

# Crea DataFrame
df = pd.DataFrame(data, columns=['Giocatore', 'Squadra', 'Gol'])
print(df)


                        Giocatore      Squadra        Gol
0                   Immobile Ciro        Lazio  27 (7 r.)
1                Martinez Lautaro        Inter  21 (2 r.)
2          Simeone Giovanni Pablo       Verona         17
3                   Abraham Tammy         Roma  17 (3 r.)
4                  Vlahovic Dusan   Fiorentina  17 (5 r.)
..                            ...          ...        ...
274  Simy Tochukwu Simeon Nwankwo  Salernitana          1
275                Amrabat Sofyan   Fiorentina          1
276               Bellanova Raoul     Cagliari          1
277                  Gondo Cedric  Salernitana          1
278              Schiavone Andrea  Salernitana          1

[279 rows x 3 columns]


In [2]:
import re
# Funzione per pulire il numero dei gol (togliere la parte tra parentesi)
def clean_gol(gol_str):
    match = re.match(r"(\d+)", gol_str)
    return int(match.group(1)) if match else None

# Applica la funzione
df['Gol Totali'] = df['Gol'].apply(clean_gol)

# Visualizza i primi risultati
print(df[['Giocatore', 'Squadra', 'Gol', 'Gol Totali']].head(10))


                 Giocatore     Squadra        Gol  Gol Totali
0            Immobile Ciro       Lazio  27 (7 r.)          27
1         Martinez Lautaro       Inter  21 (2 r.)          21
2   Simeone Giovanni Pablo      Verona         17          17
3            Abraham Tammy        Roma  17 (3 r.)          17
4           Vlahovic Dusan  Fiorentina  17 (5 r.)          17
5        Scamacca Gianluca    Sassuolo  16 (1 r.)          16
6         Berardi Domenico    Sassuolo  15 (6 r.)          15
7           Osimhen Victor      Napoli         14          14
8         Arnautovic Marko     Bologna  14 (2 r.)          14
9  Joao Pedro Guerra Cunha    Cagliari  13 (3 r.)          13


In [3]:
# Elimina la colonna "Gol"
df = df.drop(columns=['Gol'])

#aggiungo colonna anno 
df['Anno'] = 2022

# Visualizza i primi risultati per conferma
print(df.head(10))

                 Giocatore     Squadra  Gol Totali  Anno
0            Immobile Ciro       Lazio          27  2022
1         Martinez Lautaro       Inter          21  2022
2   Simeone Giovanni Pablo      Verona          17  2022
3            Abraham Tammy        Roma          17  2022
4           Vlahovic Dusan  Fiorentina          17  2022
5        Scamacca Gianluca    Sassuolo          16  2022
6         Berardi Domenico    Sassuolo          15  2022
7           Osimhen Victor      Napoli          14  2022
8         Arnautovic Marko     Bologna          14  2022
9  Joao Pedro Guerra Cunha    Cagliari          13  2022


In [4]:
#unisco i due vlahovic presenti nel dataframe e sommo i gol
df.loc[df['Giocatore'] == 'Vlahovic Dusan', 'Gol Totali'] = df.loc[df['Giocatore'] == 'Vlahovic Dusan', 'Gol Totali'].sum()

In [5]:
print(df[df['Giocatore'] == 'Vlahovic Dusan'])

         Giocatore     Squadra  Gol Totali  Anno
4   Vlahovic Dusan  Fiorentina          23  2022
51  Vlahovic Dusan    Juventus          23  2022


In [6]:
#elimino la riga con indice 4   
df = df.drop(index=4)

In [9]:
print(df.head(10))

                  Giocatore   Squadra  Gol Totali  Anno
0             Immobile Ciro     Lazio          27  2022
1          Martinez Lautaro     Inter          21  2022
2    Simeone Giovanni Pablo    Verona          17  2022
3             Abraham Tammy      Roma          17  2022
5         Scamacca Gianluca  Sassuolo          16  2022
6          Berardi Domenico  Sassuolo          15  2022
7            Osimhen Victor    Napoli          14  2022
8          Arnautovic Marko   Bologna          14  2022
9   Joao Pedro Guerra Cunha  Cagliari          13  2022
10            Pasalic Mario  Atalanta          13  2022


In [10]:
df.to_csv('marcatori_serieA2122.csv', index=False)